In [1]:
from elasticsearch import Elasticsearch

In [2]:
es = Elasticsearch("http://localhost:9200")

In [20]:
def search_videos(query: str, size: int = 5) -> list[dict]:
    """
    Search for videos whose titles or subtitles match a given query.

    This function queries a video index and returns the most relevant matches,
    giving higher importance to title matches. It also provides highlighted
    text snippets that show where the query matched within each result.

    Args:
        query (str): The text to search for in video titles and subtitles.
        size (int, optional): Maximum number of results to return. Defaults to 5.

    Returns:
        list[dict]: A list of highlighted match information for each video,
        including the video ID and highlighted snippets from the title and/or subtitles.
    """
    body = {
        "size": size,
        "query": {
            "multi_match": {
                "query": query,
                "fields": ["title^3", "subtitles"],
                "type": "best_fields",
                "analyzer": "english_with_stop_and_stem"
            }
        },
        "highlight": {
            "pre_tags": ["*"],      # highlight start
            "post_tags": ["*"],     # highlight end
            "fields": {
                "title": {
                    "fragment_size": 150,
                    "number_of_fragments": 1
                },
                "subtitles": {
                    "fragment_size": 150,
                    "number_of_fragments": 1
                }
            }
        }
    }

    response = es.search(index="podcasts", body=body)

    hits = response.body['hits']['hits']

    results = []

    for hit in hits:
        highlight = hit['highlight']
        highlight['video_id'] = hit['_id']
        results.append(highlight)

    return results
        

def get_subtitles_by_id(video_id: str) -> dict:
    """
    Retrieve the stored subtitle data for a specific video.

    Args:
        video_id (str): The unique identifier of the video.

    Returns:
        dict: The video's subtitle metadata and transcript content.
    """
    result = es.get(index="podcasts", id='1aMuynlLM3o')
    return result['_source']


In [21]:
search_videos('how do I get rich with AI?')

[{'subtitles': ['first but you still\n42:37 want to want to leverage *Ai* and you want\n42:39 to you know in introduce *AI* into\n42:43 services that your bank offers um of\n42'],
  'title': ['Trends in *AI* Infrastructure'],
  'video_id': '1aMuynlLM3o'},
 {'subtitles': [":48 responsible *AI* so how can *AI* be\n4:51 responsible yeah um definitely you know\n4:54 um it's it's a very interesting field\n4:57 and it's a very butting"],
  'title': ['Responsible and Explainable *AI*'],
  'video_id': '8Eb5mG-pC3o'},
 {'subtitles': ["prepared\n1:03 and if you're ready we can start\n1:06 okay you're ready\n1:08 yeah\n1:09 okay\n1:11 so this week we'll talk about the role\n1:13 of a staff *AI*"],
  'title': ['Staff *AI* Engineer'],
  'video_id': '_xr1_xb736E'},
 {'subtitles': ['and we have a special guest\n1:34 today BOS BOS is an *AI* and data engineer\n1:38 he specializes in moving *AI* projects\n1:40 from the good enough for demo'],
  'title': ['Data Intensive *AI*'],
  'video_id': 'BP6w_vKyS

In [22]:
from pydantic_ai import Agent

In [28]:
from pydantic_ai.messages import FunctionToolCallEvent

class NamedCallback:
    """Stream handler that prints the tool calls triggered by an agent."""

    def __init__(self, agent: Agent):
        self.agent_name = agent.name

    async def _print_function_calls(self, ctx, event) -> None:
        # Detect nested streams
        if hasattr(event, "__aiter__"):
            async for sub_event in event:  # type: ignore[attr-defined]
                await self._print_function_calls(ctx, sub_event)
            return

        if isinstance(event, FunctionToolCallEvent):
            tool_name = event.part.tool_name
            args = event.part.args
            print(f"TOOL CALL ({self.agent_name}): {tool_name}({args})")

    async def __call__(self, ctx, event) -> None:
        await self._print_function_calls(ctx, event)

In [ ]:
research_instructions = """
You're a helpful researcher agent
""".strip()

research_agent = Agent(
    name='research_agent',
    instructions=research_instructions,
    model='openai:gpt-4o-mini',
    tools=[search_videos, get_subtitles_by_id]
)

research_agent_callback = NamedCallback(research_agent)

result = await research_agent.run(
    user_prompt='how do I get rich with AI?',
    event_stream_handler=research_agent_callback
)

In [35]:
print(result.output)

Getting rich with AI involves a combination of strategies, including education, investment, entrepreneurship, and leveraging technology. Here are some steps to consider:

1. **Educate Yourself:**
   - Learn about AI fundamentals, machine learning, and data science.
   - Take online courses, attend workshops, or pursue degrees in AI-related fields.

2. **Identify Opportunities:**
   - Look for problems that AI can solve across various industries such as healthcare, finance, real estate, and more.
   - Explore niches where AI implementation can enhance efficiency, reduce costs, or create new products.

3. **Invest in AI Startups:**
   - Consider investing in AI companies that show potential for growth.
   - Keep an eye on emerging startups in the AI space through platforms like venture capital firms or crowdfunding sites.

4. **Build AI Solutions:**
   - Develop your own AI-based products or services. For example, creating software that automates tasks or analyzes data for businesses.
  

In [42]:
messages = result.new_messages()

for m in messages:
    for p in m.parts:
        kind = p.part_kind
        if kind == 'user-prompt':
            print('USER:', p.content)
            print()
        if kind == 'text':
            print('ASSISTANT:', p.content)
            print()
        if kind == 'tool-call':
            print('TOOL CALL:', p.tool_name, p.args)

USER: how do I get rich with AI?

ASSISTANT: Getting rich with AI involves a combination of strategies, including education, investment, entrepreneurship, and leveraging technology. Here are some steps to consider:

1. **Educate Yourself:**
   - Learn about AI fundamentals, machine learning, and data science.
   - Take online courses, attend workshops, or pursue degrees in AI-related fields.

2. **Identify Opportunities:**
   - Look for problems that AI can solve across various industries such as healthcare, finance, real estate, and more.
   - Explore niches where AI implementation can enhance efficiency, reduce costs, or create new products.

3. **Invest in AI Startups:**
   - Consider investing in AI companies that show potential for growth.
   - Keep an eye on emerging startups in the AI space through platforms like venture capital firms or crowdfunding sites.

4. **Build AI Solutions:**
   - Develop your own AI-based products or services. For example, creating software that automa

No tool calls

In [47]:
research_instructions = """
You're a researcher and your task is to use the available tools to conduct a research on the topic 
that the user provided.

cite the sources
""".strip()

In [ ]:


research_agent = Agent(
    name='research_agent',
    instructions=research_instructions,
    model='openai:gpt-4o-mini',
    tools=[search_videos, get_subtitles_by_id]
)

research_agent_callback = NamedCallback(research_agent)

result = await research_agent.run(
    user_prompt='how do I get rich with AI?',
    event_stream_handler=research_agent_callback
)

In [51]:
print(result.output)

{
  "stages": [
    {
      "keywords": [
        "how to get rich with AI"
      ],
      "summary": "The initial exploration of the query revealed a focus on how AI can be leveraged for personal wealth generation, including investment opportunities, starting AI-focused businesses, and utilizing AI in a wide range of industries. The search indicated various strategies and insights from experts on integrating AI technology into existing services and creating new revenue streams."
    },
    {
      "keywords": [
        "investment opportunities in AI",
        "starting AI-focused businesses",
        "AI applications in wealth generation",
        "AI for entrepreneurship",
        "AI impact on job sectors"
      ],
      "summary": "The second stage expanded upon the findings, revealing major themes around investment in AI technology and startups, as well as the economic transformation that AI brings to traditional industries. Key opinions highlighted the necessity for adaptability

In [83]:
messages = result.new_messages()

for m in messages:
    for p in m.parts:
        kind = p.part_kind
        if kind == 'user-prompt':
            print('USER:', p.content)
            print()
        if kind == 'text':
            print('ASSISTANT:', p.content)
            print()
        if kind == 'tool-call':
            print('TOOL CALL:', p.tool_name, p.args)

USER: how do I get rich with AI?

TOOL CALL: search_videos {"query":"how to get rich with AI"}
TOOL CALL: get_subtitles_by_id {"video_id":"1aMuynlLM3o"}
TOOL CALL: get_subtitles_by_id {"video_id":"8Eb5mG-pC3o"}
TOOL CALL: get_subtitles_by_id {"video_id":"_xr1_xb736E"}
TOOL CALL: get_subtitles_by_id {"video_id":"BP6w_vKySN0"}
TOOL CALL: get_subtitles_by_id {"video_id":"EQcY83VA0Us"}
TOOL CALL: get_subtitles_by_id {"video_id":"EQcY83VA0Us"}
ASSISTANT: {
  "stages": [
    {
      "keywords": [
        "how to get rich with AI"
      ],
      "summary": "The initial exploration of the query revealed a focus on how AI can be leveraged for personal wealth generation, including investment opportunities, starting AI-focused businesses, and utilizing AI in a wide range of industries. The search indicated various strategies and insights from experts on integrating AI technology into existing services and creating new revenue streams."
    },
    {
      "keywords": [
        "investment opportun

In [90]:
research_instructions = """
You are an autonomous research agent. Your goal is to perform deep, multi-stage research on the
given topic using the available search function. You must iteratively refine your understanding
of the topic and its subtopics through structured exploration.

Research process:

stage 1: initial exploration  
- Using your own knowledge of the topic, perform 3-5 broad search queries to understand the main topic
  in results and identify related areas. Only use search function, don't use anything else.
- After the initial search exploration is done, summarize key concepts, definitions, and major themes.  

stage 2: deep investigation 
- Perform 5–6 refined queries focusing on depth before doing any analysis.  
- Ispect relevant documents. Check specific mechanisms, case studies, technical details, or research gaps.  
- Gather diverse viewpoints and data to strengthen depth and accuracy.  

Rules:

1. Search queries:
   - Do not include years (e.g., “2023,” “2024”) unless explicitly part of the user’s request.
   - Always use timeless, general, or concept-based queries (e.g., “AI investment trends”
     instead of “AI investment trends 2023”).
   - Prefer queries that help build depth, context, and connections across subtopics.

2. The article must contain:
   - A clear, descriptive title.
   - An introduction with 2–3 paragraphs (each 4–6 sentences).
   - At least 10 sections, ideally 10–12, each focused on a distinct subtopic.
   - Each section must have at least 4 paragraphs, preferably 5–6.
   - Each paragraph must contain 4–6 sentences of original synthesis.

3. Each paragraph must have at least one reference object containing:
   - video_id (YouTube video ID)
   - timestamp ("mm:ss" or "h:mm:ss")
   - quote (a short excerpt or paraphrase from that video segment)
   - Do not embed citations inline in the text.

4. The conclusion must have 2–3 paragraphs summarizing the key findings and implications.

5. Structure and coherence:
   - Each section must have a unique title describing its focus.
   - Paragraphs within a section must explore different but related ideas.
   - The narrative should flow logically across sections.

6. Evidence quality:
   - All claims must be traceable to real YouTube sources.
   - References must correspond to valid video_id and timestamp pairs.
   - Quotes should reflect the video’s spoken content accurately.
   - Do not fabricate data or sources.

7. Depth and length requirements:
   - If the output has fewer than 10 sections or any section has fewer than 4 paragraphs, it is invalid.
   - If any paragraph has no references, it is invalid.
   - The report should read like an in-depth, long-form synthesis, not a short article.

8. Tone and style:
   - Write in a professional, neutral, analytical tone.
   - Emphasize explanation, synthesis, and comparison.
   - Avoid repetition and filler text.

9. Output format:
   - Markdown format
   - Populate all required fields: stages, article, sections, paragraphs, references.

10. Self-check before completion:
   - Confirm you have 10–12 sections.
   - Confirm each section has 4–6 paragraphs.
   - Confirm each paragraph has 4–6 sentences.
   - Confirm each paragraph has at least one valid reference object.
   - Confirm introduction and conclusion both contain 2–3 paragraphs.

Only output once all checks are satisfied.


Final deliverable:

The article must be long, detailed, and divided into multiple sections and paragraphs — not short summaries.

""".strip()

Get this:

```
raise ModelHTTPError(status_code=status_code, model_name=self.model_name, body=e.body) from e
pydantic_ai.exceptions.ModelHTTPError: status_code: 400, model_name: gpt-4o-mini, body: {'message': "This model's maximum context length is 128000 tokens. However, your messages resulted in 165609 tokens (165383 in the messages, 226 in the functions). Please reduce the length of the messages or functions.", 'type': 'invalid_request_error', 'param': 'messages', 'code': 'context_length_exceeded'}
```

Let's summarize our subtitles

In [72]:
sumarization_instructions = """
Your task is to summarize the provided youtube transcript for a specific topic

select the parts of the transcripts that are relevant for the topic and search qieries

Format: 

paragraph with discussion (timestamp)
""".strip()

In [73]:
sumarization_agent = Agent(
    name='summarization',
    instructions=sumarization_instructions,
    model='openai:gpt-4o-mini'
)

In [74]:
user_query = 'how do I get rich with AI?'
search_queries = [
    "investment opportunities in AI",
    "starting AI-focused businesses",
    "AI applications in wealth generation",
    "AI for entrepreneurship",
    "AI impact on job sectors"
]

subtitles = get_subtitles_by_id('1aMuynlLM3o')['subtitles']

prompt = f"""
user query:

{user_query}

search engine queries: 

{'\n'.join(search_queries)}

subtitles:

{subtitles}
""".strip()

In [75]:
print(prompt[:500])

user query:

how do I get rich with AI?

search engine queries: 

investment opportunities in AI
starting AI-focused businesses
AI applications in wealth generation
AI for entrepreneurship
AI impact on job sectors

subtitles:

0:01 everyone Welcome to our event this is
0:03 our first event in
0:05 2025 happy New Year everyone and yeah
0:08 I'm really excited to start the new year
0:10 with this stream and uh but before like
0:14 I'll do the usual thing so we have a lot
0:17 of uh things a lot of


In [76]:
summary_result = await sumarization_agent.run(prompt)

In [77]:
print(summary_result.output)

To explore ways to get rich with AI, one can look into investment opportunities, starting AI-focused businesses, and leveraging AI applications for wealth generation. Throughout the conversation, the speaker emphasized the importance of AI infrastructure and finding efficient ways to implement AI solutions, both for established companies and startups (0:37-1:41).

The discussion highlighted the growing trend of open-source AI tools and how they allow for cost-effective solutions that can give businesses a competitive edge. The guest mentioned that many enterprises are now considering how to leverage existing AI tools rather than develop their own from scratch. This approach can significantly lower operational costs while still enabling companies to tap into the benefits of AI (17:00-17:35).

Additionally, it was noted that smaller companies, which might not have extensive resources to invest heavily in infrastructure, can still thrive in the AI space by focusing on fine-tuning existing

In [86]:
from pydantic_ai import RunContext
import textwrap
import json

async def summarize(ctx: RunContext, video_id: str) -> str:
    """
    Generate a summary for a video based on the conversation history, search queries,
    and the video's subtitles.

    This function inspects prior model interaction parts to extract:
      - what the user has asked,
      - what search queries were triggered during the conversation.

    It then retrieves the subtitles of the referenced video and constructs a
    consolidated prompt containing all of this context. That prompt is sent to
    a summarization agent, and the agent's summary is returned.

    Args:
        video_id (str): The identifier of the video to summarize.

    Returns:
        str: A generated summary reflecting user intent, search behavior, and the
        video's subtitle content.
    """
    user_queries = []
    search_queries = []

    for p in m.parts:
        kind = p.part_kind
        if kind == 'user-prompt':
            user_queries.append(p.content)
        if kind == 'tool-call':
            if p.tool_name == 'search_videos':
                args = json.loads(p.args)
                query = args['query']
                search_queries.append(query)

    subtitles = get_subtitles_by_id('1aMuynlLM3o')['subtitles']
    
    prompt = textwrap.dedent(f"""
        user query:
        
        {'\n'.join(user_queries)}
        
        search engine queries: 
        
        {'\n'.join(search_queries)}
        
        subtitles:
        
        {subtitles}
    """).strip()

    summary_result = await sumarization_agent.run(prompt) 
    return summary_result.output

In [91]:


research_agent = Agent(
    name='research_agent',
    instructions=research_instructions,
    model='openai:gpt-4o-mini',
    tools=[search_videos, summarize]
)


In [92]:
research_agent_callback = NamedCallback(research_agent)

result = await research_agent.run(
    user_prompt='how do I get rich with AI?',
    event_stream_handler=research_agent_callback
)

TOOL CALL (research_agent): search_videos({"query": "how to get rich with AI", "size": 5})
TOOL CALL (research_agent): search_videos({"query": "AI investment opportunities", "size": 5})
TOOL CALL (research_agent): search_videos({"query": "building a business with AI", "size": 5})
TOOL CALL (research_agent): search_videos({"query": "AI entrepreneurship", "size": 5})
TOOL CALL (research_agent): search_videos({"query": "wealth creation with AI", "size": 5})
TOOL CALL (research_agent): summarize({"video_id": "1aMuynlLM3o"})
TOOL CALL (research_agent): summarize({"video_id": "8Eb5mG-pC3o"})
TOOL CALL (research_agent): summarize({"video_id": "_xr1_xb736E"})
TOOL CALL (research_agent): summarize({"video_id": "BP6w_vKySN0"})
TOOL CALL (research_agent): summarize({"video_id": "EQcY83VA0Us"})
TOOL CALL (research_agent): summarize({"video_id": "bT7-HRNCltk"})
TOOL CALL (research_agent): summarize({"video_id": "7Bg1JQLnCao"})
TOOL CALL (research_agent): summarize({"video_id": "pImYf9ML95Q"})
TOOL 

In [94]:
print(result.output)

# Getting Rich with AI: Strategies and Insights

Artificial Intelligence (AI) has revolutionized industries and opened new pathways to wealth creation for individuals and businesses alike. From enhancing productivity in existing jobs to creating entirely new opportunities, AI's influence is unmistakable. In exploring how to leverage AI for wealth accumulation, it is essential to understand the various avenues available, including entrepreneurial ventures, investment strategies, and incorporation into traditional business models. This article delves into the multifaceted strategies for harnessing AI's potential to achieve financial success.

## Section 1: The Evolving Landscape of AI

As AI technology continues to advance, it has become a cornerstone for innovation across sectors. Discussions about AI often center around the intricacies of AI infrastructure, which plays a crucial role in organizations adopting these technologies. For instance, a recent interview highlights that companie